In [6]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
import requests
from shapely.geometry import Point, Polygon
import dtale

In [7]:
###get Sydney Polygon
t=requests.get('https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=5750005&addressdetails='+\
               '1&hierarchy=0&group_hierarchy=1&polygon_geojson=1&format=json')
syd_poly=gpd.GeoDataFrame(t.json()['geometry'])
syd_poly['geometry']=syd_poly['coordinates'].apply(lambda y :  Polygon(y))

In [8]:
df_crash=pd.read_excel('NSW_Road_Crash_Data_2017-2021_CRASH.xlsx')
df_crash['geometry']=df_crash[['Latitude','Longitude']].apply(lambda h: Point(h[1],h[0]), axis=1)
gdf_crash=gpd.GeoDataFrame(df_crash).reset_index(drop=True)
Sydney_Crash=gdf_crash.sjoin(syd_poly, how='inner', predicate = 'within')

In [9]:
gdf_edges_attrs=pd.read_csv('unhashable_attrs_of_sydney_geo.csv')

In [10]:
gdf_edges=gpd.read_file('sydney_geo.json').merge(gdf_edges_attrs, how='left',on=['edge_id'])

In [13]:
h1=gdf_edges[['geometry','highway']].to_crs('EPSG:3857')
h1['edge_id']=np.arange(1, len(h1)+1,1)
crash_join=Sydney_Crash[['Street of crash','geometry','Crash ID']].set_crs('EPSG:4326').to_crs('EPSG:3857')\
.sjoin_nearest(h1,how='left',distance_col ='dist').to_crs('EPSG:4326')

In [14]:
gdf_edges['edge_id']=np.arange(1, len(h1)+1,1)
gdf_edges['StreetName']=gdf_edges['edge_id'].map(dict(zip(crash_join.edge_id,crash_join['Street of crash'])))
gdf_for_app=gdf_edges.dropna(subset=['StreetName']).reset_index(drop=True).dissolve(by='StreetName').reset_index(drop=True)
gdf_for_app['StreetName']=gdf_for_app['edge_id'].map(dict(zip(crash_join.edge_id,crash_join['Street of crash'])))
gdf_for_app

,geometry,oneway,length,tunnel,junction,service,edge_id,u,v,key,osmid,highway,width,access,name,lanes,ref,maxspeed,bridge,StreetName
0,"LINESTRING (151.12385 -33.88986, 151.12374 -33...",False,212.330,None,None,None,193259,1764624670,1764598368,0,170554598,residential,None,None,A'Beckett Avenue,None,None,50,None,A'BECKET
1,"LINESTRING (151.28008 -33.76355, 151.28097 -33...",False,245.204,None,None,None,140090,1835956454,5431371158,0,172717111,tertiary,None,None,Abbott Road,2,None,40,None,ABBOT
2,"LINESTRING (151.07824 -33.86575, 151.07816 -33...",False,145.974,None,None,None,198738,1809309886,1809309832,0,22496262,tertiary,None,None,Bridge Road,None,None,50,None,ABBOTSFORD
3,"MULTILINESTRING ((150.94889 -33.76738, 150.947...",True,230.025,None,None,None,3873,76466437,5124034345,0,275162541,primary,12,None,Abbott Road,2,None,60,None,ABBOTT
4,"LINESTRING (150.81759 -33.70062, 150.81682 -33...",False,71.340,None,None,None,114913,5689471779,5689471781,0,597580117,residential,None,None,Larkin Street,2,None,None,None,ABELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,"LINESTRING (151.14468 -33.90164, 151.14475 -33...",False,398.172,None,None,None,194857,1782049583,1800183754,0,"[24018001, 933092153, 931651717]",residential,None,None,Yule Street,None,None,None,None,YULE
4875,"MULTILINESTRING ((150.97773 -33.88988, 150.978...",False,109.895,None,None,None,29471,1811894556,2281645727,0,174010377,residential,None,None,Yunga Burra Street,None,None,None,None,YUNGA BURRA
4876,"LINESTRING (150.98715 -33.89925, 150.98745 -33...",False,224.845,None,None,None,181526,256225158,1847522785,0,"[373560251, 373560252]",residential,None,None,Yvonne Crescent,None,None,None,None,YVONNE
4877,"LINESTRING (150.92510 -33.79495, 150.92468 -33...",False,118.843,None,None,None,49928,1854420708,1854420755,0,317590117,residential,None,None,Zermatt Avenue,None,None,None,None,ZERMATT


In [16]:
gdf_for_app.to_file("app_geo.json", driver="GeoJSON")

In [38]:
Sydney_Crash.reset_index(drop=True).drop(columns=['geometry','coordinates']).to_csv('SydneyCrash.csv', index=False)